In [ ]:
import pandas as pd


folder = "./Data/bybit"
file = "ETH-USDT:USDT_1m.csv"

features_base = ['timestamp', 'close']
features_stochrsi = ['stoch_rsi_overbought', 'stoch_rsi_oversold', 'stoch_rsi_bullish_crossover', 'stoch_rsi_bearish_crossover', 'stoch_rsi_trend', 'stoch_rsi_change_of_trend']
features_hma = ['hma200_trend', 'hma200_change_of_trend']
features_macd = ['macd_bullish_crossover', 'macd_bearish_crossover', 'macd_positive', 'macd_negative', 'macd_bullish_crossover_positive', 'macd_bearish_crossover_negative', 'macd_trend', 'macd_change_of_trend']
columns = features_base + features_stochrsi + features_hma + features_macd

periods = [5, 10, 15, 20, 30, 60, 120]

df = pd.read_csv(f"{folder}/{file}")



Dataframe must be indexed by timestamp

In [ ]:
df = df[columns]
df = df.set_index('timestamp')

df.tail(10)

Now we calculate the result for different number of intervals

In [ ]:
# adding shifted periods results
features_close = [f'close_in_{period}' for period in periods]
features_close_perc = [f'close_p_in_{period}' for period in periods]
for period in periods:
    column_name = f'close_in_{period}'
    df[column_name] = df['close'].shift(-period)
    column_perc = f'close_p_in_{period}'
    df[column_perc] = ((df[column_name] - df['close']) / df['close']) * 100
    
df.columns

we classify the features based on the result for the given number of intervals

In [ ]:
# Create a column variable that is the combination of all the features_ lists
all_features = features_stochrsi + features_hma + features_macd

period = 10

# removing likely not valid rows
features = all_features + [f'close_p_in_{period}']
new_df = df[features].iloc[50:-period]

# grouped_df = new_df.groupby(all_features)[f'close_p_in_{period}'].agg(['count', 'mean', 'min', 'max'])
# grouped_df = grouped_df.sort_values(by='mean', ascending=False)

# grouped_df.head(10)

In [ ]:
def get_best_group(df, features, period, sort='mean'):
    grouped_df = df.groupby(features)[f'close_p_in_{period}'].agg(['count', 'mean', 'min', 'max'])
    grouped_df = grouped_df.sort_values(by=sort, ascending=False)
    return grouped_df.head(5)


def convert_grouped_to_dataframe(grouped_df):
    df = grouped_df.reset_index()
    return df


In [ ]:

get_best_group(new_df, features_stochrsi, period)

In [ ]:

get_best_group(new_df, features_hma, period)

In [ ]:

get_best_group(new_df, features_macd, period)

In [ ]:

get_best_group(new_df, features_stochrsi+features_macd, period)

In [22]:
folder_lab1 = "./Data/lab1"
features_to_test = features_stochrsi
for period in periods:
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    result = get_best_group(new_df, features_to_test, period, 'count')
    result.to_csv(f"{folder_lab1}/{file}_stochrsi_{period}.csv")
    print(result)
    
    # print(get_best_group(new_df, features_hma, period))
    # print(get_best_group(new_df, features_macd, period, 'count'))
    # print(get_best_group(new_df, features_stochrsi+features_macd, period))
    print("")

Period: 5
                                                                                                                                           count  \
stoch_rsi_overbought stoch_rsi_oversold stoch_rsi_bullish_crossover stoch_rsi_bearish_crossover stoch_rsi_trend stoch_rsi_change_of_trend          
False                False              False                       False                       Uptrend         Downtrend->Uptrend            25   
                                                                                                Downtrend       Uptrend->Downtrend            21   
                     True               False                       False                       Uptrend         Downtrend->Uptrend            12   
True                 False              False                       False                       Downtrend       Uptrend->Downtrend            10   
False                True               False                       False                       Downtr

i want to find the best combination of features, maximising the results

In [ ]:
import pandas as pd

def append_dataframes(df1, df2):
    # Concatenate the dataframes
    df = pd.concat([df1, df2], ignore_index=True)
    return df


now we find the combo of features that are performing better

In [24]:
import itertools

features_to_test = features_stochrsi + features_hma

combinations_stochrsi = []
for r in range(1, len(features_to_test) + 1):
    combinations_stochrsi.extend(itertools.combinations(features_to_test, r))
 
print(len(combinations_stochrsi))   
print(combinations_stochrsi)

for period in periods:
    analysis_df = pd.DataFrame()
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    for combination in combinations_stochrsi:
        result = get_best_group(new_df, list(combination), period, 'count')
        tmp_df = convert_grouped_to_dataframe(result)
        tmp_df['period'] = period
        analysis_df = append_dataframes(tmp_df,analysis_df)
        # print(result)
        print("")
    analysis_df.to_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

255
[('stoch_rsi_overbought',), ('stoch_rsi_oversold',), ('stoch_rsi_bullish_crossover',), ('stoch_rsi_bearish_crossover',), ('stoch_rsi_trend',), ('stoch_rsi_change_of_trend',), ('hma200_trend',), ('hma200_change_of_trend',), ('stoch_rsi_overbought', 'stoch_rsi_oversold'), ('stoch_rsi_overbought', 'stoch_rsi_bullish_crossover'), ('stoch_rsi_overbought', 'stoch_rsi_bearish_crossover'), ('stoch_rsi_overbought', 'stoch_rsi_trend'), ('stoch_rsi_overbought', 'stoch_rsi_change_of_trend'), ('stoch_rsi_overbought', 'hma200_trend'), ('stoch_rsi_overbought', 'hma200_change_of_trend'), ('stoch_rsi_oversold', 'stoch_rsi_bullish_crossover'), ('stoch_rsi_oversold', 'stoch_rsi_bearish_crossover'), ('stoch_rsi_oversold', 'stoch_rsi_trend'), ('stoch_rsi_oversold', 'stoch_rsi_change_of_trend'), ('stoch_rsi_oversold', 'hma200_trend'), ('stoch_rsi_oversold', 'hma200_change_of_trend'), ('stoch_rsi_bullish_crossover', 'stoch_rsi_bearish_crossover'), ('stoch_rsi_bullish_crossover', 'stoch_rsi_trend'), ('sto

In [29]:
period = 60
analysis_df = pd.read_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

# Filter the dataframe to include only rows where the count is greater than 1
filtered_df = analysis_df[analysis_df['count'] > 1]

# I want to minimise the losses
filtered_df = filtered_df[filtered_df['mean'] > filtered_df['min'].abs()]

# Sort the dataframe by mean in descending order and by the difference between min and max in ascending order
sorted_df = filtered_df.sort_values(by=['mean', 'max', 'min'], ascending=[False, True, True])

sorted_df.head(5)


,Unnamed: 0,stoch_rsi_overbought,stoch_rsi_oversold,stoch_rsi_bullish_crossover,stoch_rsi_bearish_crossover,stoch_rsi_trend,stoch_rsi_change_of_trend,hma200_trend,hma200_change_of_trend,count,mean,min,max,period
391,391,NaN,True,False,NaN,NaN,Downtrend->Uptrend,Downtrend,NaN,6,0.308050,-0.043987,0.870984,60
30,30,NaN,False,False,False,Downtrend,Uptrend->Downtrend,Downtrend,NaN,10,0.272472,-0.229046,0.915865,60
145,145,NaN,False,False,False,NaN,Uptrend->Downtrend,Downtrend,NaN,10,0.272472,-0.229046,0.915865,60
15,15,False,False,False,False,Downtrend,Uptrend->Downtrend,Downtrend,NaN,6,0.269511,-0.229046,0.915865,60
184,184,False,NaN,False,NaN,Uptrend,Downtrend->Uptrend,Downtrend,NaN,16,0.247166,-0.226512,0.982617,60
